# 🤖 AI Assistant Colab Backend - Batch Mode with Assistant Loader

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os

CONFIG_DIR = '/content/drive/MyDrive/assistant_configs'
OUTPUT_DIR = '/content/drive/MyDrive/assistant_outputs'

os.makedirs(CONFIG_DIR, exist_ok=True)
os.makedirs(OUTPUT_DIR, exist_ok=True)
print('✅ Folders are ready.')

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime

def run_web_scraper(config):
    url = config.get('url', '')
    filters = [f.strip() for f in config.get('filters', '').split(',') if f.strip()]

    print(f'🌐 Scraping: {url}')
    try:
        response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
        soup = BeautifulSoup(response.text, 'html.parser')
        items = []
        for item in soup.select('.item-cell'):
            title = item.select_one('.item-title')
            price = item.select_one('.price-current')
            rating = item.select_one('.item-rating')

            data = {
                'title': title.text.strip() if title else None,
                'price': price.text.strip() if price else None,
                'rating': rating['title'] if rating and rating.has_attr('title') else None
            }
            filtered = {k: v for k, v in data.items() if k in filters or not filters}
            items.append(filtered)

        df = pd.DataFrame(items)
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        filename = f"web_scraper_output_{timestamp}.csv"
        output_path = os.path.join(OUTPUT_DIR, filename)
        df.to_csv(output_path, index=False)
        print(f'✅ Scraped {len(items)} items | Saved to {output_path}')
    except Exception as e:
        print(f'❌ Error scraping {url}: {e}')

def run_api_fetcher(config):
    print(f"⚙️ Running API Fetcher (Placeholder)")

In [ ]:
# 🔁 Batch Mode Toggle
batch_mode = True

# Process each config in folder
config_files = [f for f in os.listdir(CONFIG_DIR) if f.endswith('.json')]
if not config_files:
    raise FileNotFoundError('❌ No config files found.')

for file in config_files:
    config_path = os.path.join(CONFIG_DIR, file)
    print(f'\n📄 Processing config: {file}')
    try:
        with open(config_path, 'r') as f:
            config = json.load(f)
        task_type = config.get('task_type')
        if task_type == 'web_scraper':
            run_web_scraper(config)
        elif task_type == 'api_fetcher':
            run_api_fetcher(config)
        else:
            print(f'⚠️ Unknown task_type: {task_type}')
    except Exception as e:
        print(f'❌ Failed to process {file}: {e}')